In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime
import time
from selenium.webdriver.common.keys import Keys
import re
import csv

In [29]:
def is_xpath_present(driver, xpath):
    return len(driver.find_elements(By.XPATH, xpath)) > 0

In [31]:
    # 対象競馬場とレース
    place_codes = [
        "35", "45", "55", "65", "46", "54", "44",
        "47", "50", "30", "36", "43", "48", "42"
    ]
    # レース番号
    race_numbers = [f"{i:02d}" for i in range(1, 13)]

In [37]:
def login_netkeiba():
    
    service = Service('C:\\Users\\ryo\\AppData\\Local\\SeleniumBasic\\chromedriver.exe')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    driver = webdriver.Chrome(service=service, options=options)

    # ログイン
    url_login = "https://regist.netkeiba.com/account/?pid=login"
    driver.get(url_login)
    driver.find_element(By.NAME, "login_id").send_keys("keibaaitool@gmail.com")
    pw_field = driver.find_element(By.NAME, "pswd")
    pw_field.send_keys("keibaai")
    pw_field.send_keys(Keys.ENTER)
    time.sleep(3)

    # 日付
    today = datetime.now()
    yyyy = today.strftime("%Y")
    mmdd = today.strftime("%m%d")
    race_date = today.strftime("%Y-%m-%d")

    # CSVファイルを準備（追記モード）
    with open('C:\\Users\\ryo\Downloads\\race_data.csv', mode='w', newline='', encoding='cp932') as csvfile:
        writer = csv.writer(csvfile)
        # ヘッダーを書き込む
        writer.writerow([
            "race_id", "race_date", "racecourse", "race_number", "ground", "distance", "direction",
            "num_horses", "race_time", "horse_number", "horse_name", "horse_id",
            "jockey_name", "jockey_id"
        ])

        for place_code in place_codes:
            for race_num in race_numbers:
                race_id = f"{yyyy}{place_code}{mmdd}{race_num}"
                url = f"https://nar.netkeiba.com/race/shutuba.html?race_id={race_id}"
                driver.get(url)
                time.sleep(1)
    
                try:
                    # 競馬場
                    racecourse_elem = driver.find_element(By.CSS_SELECTOR,
                        "#Netkeiba_Race_Nar_Shutuba > div.Wrap.fc > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData02 > span:nth-child(2)")
                    racecourse = racecourse_elem.text
                    # R
                    race_number_elem = driver.find_element(By.CSS_SELECTOR,
                        "#Netkeiba_Race_Nar_Shutuba > div.Wrap.fc > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item01 > div")
                    race_number = race_number_elem.text
                    # 頭数
                    num_horses_elem = driver.find_element(By.CSS_SELECTOR,
                        "#Netkeiba_Race_Nar_Shutuba > div.Wrap.fc > div.RaceColumn01 > div > div.RaceMainColumn > div.RaceList_NameBox > div.RaceList_Item02 > div.RaceData02 > span:nth-child(5)")
                    num_horses_text = num_horses_elem.text
                    num_horses = int(num_horses_text.replace("頭", "").strip())
                    # レース情報（まとめられている）
                    race_info_xpath = '//*[@id="Netkeiba_Race_Nar_Shutuba"]/div[1]/div[2]/div/div[1]/div[3]/div[2]/div[2]'
                    race_info_elem = driver.find_element(By.XPATH, race_info_xpath)
                    race_info_text = race_info_elem.text.strip()
                    # "/"で分割
                    parts = [p.strip() for p in race_info_text.split("/")]            
                    # 発送時刻を抽出
                    race_time = parts[0].replace("発走", "").strip()
                    ground_info = parts[1]
                    # グラウンドを抽出
                    ground_match = re.search(r"(芝|ダ)", ground_info)
                    ground = ground_match.group(1) if ground_match else "不明"
                    # 距離を抽出
                    distance_match = re.search(r"(\d+)m", ground_info)
                    distance = int(distance_match.group(1)) if distance_match else -1
                    # 向きを抽出
                    direction_match = re.search(r"\((左|右)\)", ground_info)
                    direction = direction_match.group(1) if direction_match else "不明"
                    # 出力確認
                    print(f"\n📄 Race ID: {race_id} | 競馬場: {racecourse} | R: {race_number} | 頭数: {num_horses} | 🕓 発走時刻: {race_time} | 🏇 グラウンド: {ground} | 📏 距離: {distance}m | ↩ 回り: {direction}")
                    # 馬番・馬名取得
                    # 馬番・馬名・馬名リンク取得
                    for i in range(1, num_horses + 1):
                        try:
                            # 馬番
                            horse_number_xpath = f'//*[@id="tr_{i}"]/td[2]'
                            horse_number = driver.find_element(By.XPATH, horse_number_xpath).text.strip()
                            # 馬名と馬ID
                            horse_link_xpath_1 = f'/html/body/div[1]/div[3]/div[3]/table/tbody/tr[{i}]/td[4]/div/div/span[2]/a'
                            horse_link_xpath_2 = f'/html/body/div[1]/div[3]/div[3]/table/tbody/tr[{i}]/td[4]/div/div/span/a'
                            if is_xpath_present(driver, horse_link_xpath_1):
                                horse_link_xpath = horse_link_xpath_1
                            elif is_xpath_present(driver, horse_link_xpath_2):
                                horse_link_xpath = horse_link_xpath_2
                            horse_link_elem = driver.find_element(By.XPATH, horse_link_xpath)
                            horse_name = horse_link_elem.text.strip()
                            horse_href = horse_link_elem.get_attribute("href")
                            horse_id = horse_href.rstrip("/").split("/")[-1]
                            # 騎手名と騎手ID
                            jockey_link_xpath = f'//*[@id="tr_{i}"]/td[7]/span/a'
                            jockey_link_elem = driver.find_element(By.XPATH, jockey_link_xpath)
                            jockey_name = jockey_link_elem.text.strip()
                            jockey_href = jockey_link_elem.get_attribute("href")
                            jockey_id = jockey_href.rstrip("/").split("/")[-1]
                            print(f"  馬番{i}: {horse_number} | 馬名: {horse_name} | 馬ID: {horse_id} | 騎手名: {jockey_name} | 騎手ID: {jockey_id}")
    
                            writer.writerow([
                                race_id,
                                race_date,
                                racecourse,
                                race_number,
                                ground,
                                distance,
                                direction,
                                num_horses,
                                race_time,
                                horse_number,
                                horse_name,
                                horse_id,
                                jockey_name,
                                jockey_id
                            ])
                        
                        except Exception as e:
                            print(f"  馬番{i}: データ取得失敗 ({e})")
    
                except Exception as e:
                    print(f"❌ Race ID: {race_id} データ取得に失敗: {e}")
                    break
    driver.quit()

login_netkeiba()


<>:24: SyntaxWarning: invalid escape sequence '\D'
<>:24: SyntaxWarning: invalid escape sequence '\D'
C:\Users\ryo\AppData\Local\Temp\ipykernel_8632\1789349852.py:24: SyntaxWarning: invalid escape sequence '\D'
  with open('C:\\Users\\ryo\Downloads\\race_data.csv', mode='w', newline='', encoding='cp932') as csvfile:


❌ Race ID: 202535101501 データ取得に失敗: invalid literal for int() with base 10: ''

📄 Race ID: 202545101501 | 競馬場: 川崎 | R: 1R | 頭数: 12 | 🕓 発走時刻: 14:45 | 🏇 グラウンド: ダ | 📏 距離: 900m | ↩ 回り: 左
  馬番1: 1 | 馬名: スマートドラフト | 馬ID: 2019101309 | 騎手名: 赤津和希 | 騎手ID: 05561
  馬番2: 2 | 馬名: パトウォン | 馬ID: 2019103836 | 騎手名: 櫻井光輔 | 騎手ID: 05563
  馬番3: 3 | 馬名: ササキンポピー | 馬ID: 2021100463 | 騎手名: 古岡勇樹 | 騎手ID: 05606
  馬番4: 4 | 馬名: ダイナマイトブル | 馬ID: 2020101068 | 騎手名: 達城龍次 | 騎手ID: 05540
  馬番5: 5 | 馬名: シンキングセンティ | 馬ID: 2019100703 | 騎手名: 岡田大 | 騎手ID: 05348
  馬番6: 6 | 馬名: ビギニングドリーム | 馬ID: 2018102352 | 騎手名: 神尾香澄 | 騎手ID: 05613
  馬番7: 7 | 馬名: ハナサキポプラ | 馬ID: 2020100441 | 騎手名: 佐野遥久 | 騎手ID: a0570
  馬番8: 8 | 馬名: カナサンドー | 馬ID: 2021101746 | 騎手名: 藤江渉 | 騎手ID: a0188
  馬番9: 9 | 馬名: ゴールドボード | 馬ID: 2019100826 | 騎手名: 秋元耕成 | 騎手ID: 05347
  馬番10: 10 | 馬名: カラルナ | 馬ID: 2020100869 | 騎手名: 伊藤裕人 | 騎手ID: a0257
  馬番11: 11 | 馬名: リオシャンパーニュ | 馬ID: 2019105561 | 騎手名: 笹川翼 | 騎手ID: 05524
  馬番12: 12 | 馬名: セキノアトラス | 馬ID: 2018104269 | 騎手名: 岡村裕基 | 騎手ID: 05272

📄 Race ID